Para la realización de este trabajo se ha usado exclusivamente la librería Opencv y la gran mayoría de funcionalidades que esta posee.

In [1]:
import cv2 as cv 
import numpy as np
#CV.RESHAPE
# Función para redimensionar el video (si es necesario)
def rescale(frame, scale=0.5):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    return cv.resize(frame, (width, height), interpolation=cv.INTER_AREA)

Como el video presenta una resolución muy grande para presentarlo en modo ventana (1920x1080) lo primero que haremos será definir la función `rescale`, para así reescalar la resolución del video y poder visualizarlo con normalidad en cualquier ordenador. El resultado después de aplicar la función sera una resolución multiplicada por el valor `scale` que en ese caso es 0.75.


Posterior a la creación de la función `rescale` leemos el archivo de video con la función `VideoCapture`, para posteriormente crear la referencia a la función 'createBackgroundSubtractorMOG2' llamada `object_detector`esto nos servirá para hacer una detección del  movimiento eliminando el fondo de un video. Usa un método adaptativo que actualiza el fondo dinámicamente para reconocer objetos nuevos o en movimiento. 

Ahora definiremos las variables más importantes para el conteo de los coches,  siendo `line_position` la altura a la que estaríá la barra desde donde contamos los coches; carril_min_x y carril_max_x los indicadores que nos permiten saber cual será el carril que vamos a contar.

In [2]:
import cv2 as cv
import numpy as np

# Capturando el video
video = cv.VideoCapture('trafico.mp4')
object_detector = cv.createBackgroundSubtractorMOG2(history=100, varThreshold=50)  # Ajustar el umbral

# Variables para conteo
count = 0
line_position = 280  # Ajustar la posición de la línea (relativa al ROI)
car_ids = set()
tracking_objects = {}
next_car_id = 0

# Límites del carril (modificar según posición del carril en el ROI)
carril_min_x = 0
carril_max_x = 600

Ahora para vizualizar el video usaremos un bucle while, con él podremos imprimir por pantalla todos los frames además de hacer las modificaciones y estimaciones pertinentes para saber si pasan vehículos por la zona indicada.

Primero reescalamos el frame, dado que es muy grande para mostrarlo por una pantalla normal. Posteriormente crearemos la zona de interes `roi` después aplicamos el `object_detector`, además de eso le aplicamos el `medianBlur` que funciona para elimina el ruido al reemplazando cada píxel con la mediana de los píxeles en su vecindad, y por último el `threshold` que sirve para convertir una imagen en escala de grises en una binaria, todo esto es necesario para limpiar la mascara para que los objetos sean más definidos.

Posteriormente detectaremos los contornos en una máscara binaria usando `cv.findContours`, filtra los contornos pequeños (área menor a 800) para evitar ruido y calcula un rectángulo delimitador (`cv.boundingRect`) para cada contorno relevante, obteniendo su centroide (coordenadas cx, cy) como representación del centro del objeto detectado.

Con (`tracking_objects`)  ratreamos objetos previamente detectados comparando sus posiciones anteriores `prev_center` con la posición actual del centroide. Si la distancia entre ambos es menor a 50 píxeles, se considera que es el mismo objeto, se actualiza su posición en `current_objects` y se marca como coincidente `found_match = True`.

In [ ]:
import cv2 as cv
import numpy as np

# Renderizando cada frame
while True:
    state, frame = video.read()
    if not state:
        break

    frame = rescale(frame)
    roi = frame[200:725, 200:1300] # Definir ROI (ajustar según el video), y = 200:725, x = 200:1300
    mask = object_detector.apply(roi) # Aplicar el detector de objetos, de la librería cv2
    cv.imshow("mascara", mask) # Mostrar la máscara para depuración
    mask = cv.medianBlur(mask, 5) # Filtro para reducir ruido, medianblur significa que el píxel se reemplaza por la mediana de los píxeles en su vecindad
    mask = cv.threshold(mask, 127, 255, cv.THRESH_BINARY)[1]  # Binarización para mejor detección, thresholding convierte la imagen en una imagen binaria y los parametros son: fuente, umbral, valor máximo y tipo de umbral, thresh_binary significa que los píxeles mayores que el umbral se establecen en el valor máximo y los menores en 0

    contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE) # Encontrar contornos de los objetos detectados en la máscara, retr_tree significa que se recuperan todos los contornos y se reconstruye la jerarquía completa, chain_approx_simple significa que se almacena solo los puntos finales de las líneas
    current_objects = {}

    for contour in contours:
        area = cv.contourArea(contour) # Calcular el área del contorno 
        if area > 800:  # Ajustar el área mínima para evitar falsos positivos
            x, y, w, h = cv.boundingRect(contour) # Obtener el rectángulo delimitador del contorno 
            cx, cy = int(x + w / 2), int(y + h / 2) # Calcular el centroide del rectángulo 

            # Verificar si el coche está dentro del carril
            if carril_min_x <= cx <= carril_max_x:
                # Dibuja el contorno y centroide
                cv.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv.circle(roi, (cx, cy), 5, (0, 0, 255), -1)

                # Asignar un ID al coche si no está siendo rastreado
                found_match = False
                for car_id, prev_center in tracking_objects.items():
                    prev_x, prev_y = prev_center
                    if abs(cx - prev_x) < 50 and abs(cy - prev_y) < 50:
                        current_objects[car_id] = (cx, cy)
                        found_match = True

                        # Verificar si cruzó la línea
                        if prev_y < line_position <= cy:
                            if car_id not in car_ids:
                                count += 1
                                car_ids.add(car_id)

                if not found_match:
                    current_objects[next_car_id] = (cx, cy)
                    next_car_id += 1

    tracking_objects = current_objects

    # Dibujar la línea de conteo y los límites del carril
    cv.line(roi, (0, line_position), (roi.shape[1], line_position), (255, 0, 0), 2)
    cv.line(roi, (carril_min_x, 0), (carril_min_x, roi.shape[0]), (0, 255, 255), 2)
    cv.line(roi, (carril_max_x, 0), (carril_max_x, roi.shape[0]), (0, 255, 255), 2)

    # Dibujar trayectorias de los coches
    for car_id, center in tracking_objects.items():
        cx, cy = center
        cv.putText(roi, str(car_id), (cx, cy - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)
        cv.circle(roi, (cx, cy), 3, (255, 255, 0), -1)

    # Mostrar el conteo en pantalla
    cv.putText(frame, f'Car count: {count}', (50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 100, 0), 2)

    cv.imshow('Trafico', frame)
    cv.imshow('ROI', roi)
    cv.imshow('Mask', mask)

    if cv.waitKey(10) & 0xFF == ord('d'):
        break

video.release()
cv.destroyAllWindows()


`line_position` comparando su posición previa `prev_y` con su posición actual `cy` para saber si el objeto cruzo la línea. Si cruza la línea y su car_id no está registrado, incrementa un contador y lo añade a un conjunto de IDs `car_ids`. Si no se encuentra una coincidencia con un objeto existente, se asigna un nuevo ID al objeto, se registra en `current_objects`, y el ID se incrementa. Finalmente, actualiza los objetos en seguimiento  y marca que se encontró una coincidencia `found_match`.

Y por último con `line_position` dibujamos la línea del conteo y dos líneas verticales que delimitan los carriles `carril_min_x` y `carril_max_x` sobre la región de interés. Luego, para cada coche rastreado, dibuja su trayectoria marcando su posición actual con un círculo y su ID con texto cercano al centro del objeto. Finalmente, muestra en la ventana principal el conteo total de coches detectados usando texto en la parte superior de la pantalla.

In [ ]:
import cv2 as cv
import numpy as np
from collections import deque

# --- 1. Funciones Auxiliares ---

def rescale(frame, scale=0.5):
    """Reescala el frame manteniendo la proporción."""
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    return cv.resize(frame, (width, height), interpolation=cv.INTER_AREA)

# --- 2. Configuración Inicial ---

# Capturando el video (asegúrate de que 'trafico.mp4' esté en la misma carpeta)
video = cv.VideoCapture('trafico.mp4')
if not video.isOpened():
    print("Error: No se pudo abrir el video.")
    exit()

# Configuración del detector de fondo MOG2
object_detector = cv.createBackgroundSubtractorMOG2(
    history=500,
    varThreshold=25,
    detectShadows=True
)

# Variables para conteo y seguimiento
car_count = 0
car_ids = set()
tracking_objects = {}  # {ID: (cx, cy)} almacena el centroide actual
next_car_id = 0

# Configuración específica para el video
SCALE = 0.75  # Reescalar a 75%
LINE_POSITION_Y = int(450 * SCALE)  # Posición de la línea de conteo
MIN_CAR_AREA = 1000  # Área mínima para ser considerado un coche
MAX_DISTANCE = 70  # Distancia máxima entre centroides para el mismo coche

# --- 3. Bucle Principal de Procesamiento ---

while True:
    ret, frame = video.read()
    if not ret:
        print("Fin del video o error de lectura.")
        break

    # Reescalar el frame completo
    frame = rescale(frame, scale=SCALE)
    height, width = frame.shape[:2]

    # 1. Aplicar detector de fondo para obtener la máscara de movimiento
    mask = object_detector.apply(frame)

    # 2. Filtrado Morfológico (para mejorar la máscara)
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel, iterations=1)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel, iterations=1)

    # 3. Detección de contornos
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Diccionario para almacenar los objetos detectados en el frame actual
    current_objects = {}

    # 4. Procesar contornos y aplicar filtros
    for contour in contours:
        area = cv.contourArea(contour)
        if area < MIN_CAR_AREA:
            continue

        x, y, w, h = cv.boundingRect(contour)
        cx, cy = x + w // 2, y + h // 2  # Centroide

        # (Opcional) Filtro de proporción (aspect ratio)
        aspect_ratio = w / h
        if aspect_ratio > 4.0 or aspect_ratio < 0.5:
            continue

        # 5. Seguimiento por centroides
        found_match = False
        car_id_matched = -1

        for car_id, prev_center in tracking_objects.items():
            prev_x, prev_y = prev_center
            distance = np.sqrt((cx - prev_x)**2 + (cy - prev_y)**2)

            if distance < MAX_DISTANCE:
                car_id_matched = car_id
                found_match = True
                break

        if found_match:
            current_objects[car_id_matched] = (cx, cy)

            # 6. Lógica de conteo (si cruza la línea)
            if tracking_objects[car_id_matched][1] < LINE_POSITION_Y <= cy:
                if car_id_matched not in car_ids:
                    car_count += 1
                    car_ids.add(car_id_matched)
        else:
            # Nuevo objeto
            current_objects[next_car_id] = (cx, cy)
            next_car_id += 1

        # 7. Dibujar el contorno verde con el ID
        final_id = car_id_matched if found_match else next_car_id - 1
        cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.putText(frame, f'ID: {final_id}', (x, y - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # Actualizar los objetos en seguimiento
    tracking_objects = current_objects

    # 8. Dibujar la línea de conteo
    cv.line(frame, (0, LINE_POSITION_Y), (width, LINE_POSITION_Y), (255, 0, 0), 2)

    # 9. Mostrar el conteo en pantalla
    cv.putText(frame, f'Car count: {car_count}', (50, 50),
               cv.FONT_HERSHEY_SIMPLEX, 1, (255, 100, 0), 2)

    # Mostrar resultados
    cv.imshow('Trafico - Deteccion y Tracking', frame)
    cv.imshow('Mascara de Movimiento Filtrada', mask)

    # Salir con la tecla 'd'
    if cv.waitKey(10) & 0xFF == ord('d'):
        break

# Liberar recursos
video.release()
cv.destroyAllWindows()
